Notebook to test the models used for FL in a traditional approach to check their performance.

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.models import EfficientNet_B0_Weights
from torch.utils.data import DataLoader

batch_size = 1024

# Define transformations
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Define the EfficientNet-B0 model
def get_model():
    weights = EfficientNet_B0_Weights.IMAGENET1K_V1
    model = models.efficientnet_b0(weights=weights)
    # Modify classifier head for CIFAR-10 (10 output classes)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 10)
    return model

model = get_model()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to evaluate the model on the test set
def evaluate(model, testloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# Train and evaluate the model
num_epochs = 20  # You can adjust the number of epochs
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(trainloader.dataset)
    accuracy = evaluate(model, testloader, device)
    print(f'[{epoch + 1}/{num_epochs}] Test Accuracy: {accuracy:.2f}%,\t loss: {epoch_loss:.4f}')

print('Finished Training')

In [5]:
def print_model_parameters(net):
    total_params = sum(p.numel() for p in net.parameters())
    trainable_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
print_model_parameters(model)

MNIST
===

from https://medium.com/@deepeshdeepakdd2/lenet-5-implementation-on-mnist-in-pytorch-c6f2ee306e37

In [8]:
from models.lenet5 import LeNet5

In [9]:
# Define transformations for MNIST
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))]) # Mean and std for MNIST

# Load MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2, drop_last=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Instantiate the LeNet-5 model
model = LeNet5()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print_model_parameters(model)

Total parameters: 61,706
Trainable parameters: 61,706


In [12]:
# Train and evaluate the model
num_epochs = 20  # You can adjust the number of epochs
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(trainloader.dataset)
    accuracy = evaluate(model, testloader, device)
    print(f'[{epoch + 1}/{num_epochs}] Test Accuracy: {accuracy:.2f}%,\t loss: {epoch_loss:.4f}')

print('Finished Training')

[1/20] Test Accuracy: 89.41%,	 loss: 1.0143
[2/20] Test Accuracy: 93.67%,	 loss: 0.2995
[3/20] Test Accuracy: 95.42%,	 loss: 0.1925
[4/20] Test Accuracy: 96.71%,	 loss: 0.1353
[5/20] Test Accuracy: 97.22%,	 loss: 0.1009
[6/20] Test Accuracy: 97.84%,	 loss: 0.0801
[7/20] Test Accuracy: 98.18%,	 loss: 0.0660
[8/20] Test Accuracy: 98.35%,	 loss: 0.0557
[9/20] Test Accuracy: 98.35%,	 loss: 0.0490
[10/20] Test Accuracy: 98.49%,	 loss: 0.0431
[11/20] Test Accuracy: 98.64%,	 loss: 0.0386
[12/20] Test Accuracy: 98.62%,	 loss: 0.0349
[13/20] Test Accuracy: 98.68%,	 loss: 0.0310
[14/20] Test Accuracy: 98.82%,	 loss: 0.0287
[15/20] Test Accuracy: 98.75%,	 loss: 0.0263
[16/20] Test Accuracy: 98.87%,	 loss: 0.0237
[17/20] Test Accuracy: 98.89%,	 loss: 0.0215
[18/20] Test Accuracy: 98.88%,	 loss: 0.0188
[19/20] Test Accuracy: 98.98%,	 loss: 0.0184
[20/20] Test Accuracy: 98.96%,	 loss: 0.0168
Finished Training
